## CTU-13 data 전처리 과정


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd
dataset=pd.read_parquet('./drive/MyDrive/캡스톤/dataset/10-Rbot-20110818.binetflow.parquet')

In [ ]:
dataset

,dur,proto,dir,state,stos,dtos,tot_pkts,tot_bytes,src_bytes,label,Family
0,1.060248,tcp,->,S_RA,0.0,0.0,4,252,132,flow=Background-TCP-Attempt,20110818.binetflow.csv
1,279.349152,tcp,<?>,SRPA_PA,0.0,0.0,15,1318,955,flow=Background-TCP-Attempt,20110818.binetflow.csv
2,166.390015,tcp,->,SR_SA,0.0,0.0,3,212,134,flow=Background-TCP-Established,20110818.binetflow.csv
3,1.187083,tcp,->,S_RA,0.0,0.0,4,244,124,flow=Background-TCP-Attempt,20110818.binetflow.csv
4,0.980571,tcp,->,S_RA,0.0,0.0,4,244,124,flow=Background-TCP-Attempt,20110818.binetflow.csv
...,...,...,...,...,...,...,...,...,...,...,...
776092,0.109055,tcp,->,SPA_SA,0.0,0.0,5,557,417,flow=From-Normal-V51-Stribrek,20110818.binetflow.csv
776093,0.038428,tcp,->,SPA_SPA,0.0,0.0,6,1638,1008,flow=Background-TCP-Established,20110818.binetflow.csv
776094,0.000158,tcp,->,S_SA,0.0,0.0,2,152,78,flow=Background-TCP-Established,20110818.binetflow.csv
776095,0.000688,udp,<->,CON,0.0,0.0,2,306,244,flow=Background-UDP-Established,20110818.binetflow.csv


In [ ]:
# 총 9개의 feature
# categorical 변수 - proto, dir, state, stos, dtos
# Numerical 변수 - dur, tot_pkts, tot_bytes, src_bytes
# label -> Background, Normal, Botnet
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 776097 entries, 0 to 776096
Data columns (total 11 columns):
 #   Column     Non-Null Count   Dtype   
---  ------     --------------   -----   
 0   dur        776097 non-null  float32 
 1   proto      776097 non-null  category
 2   dir        776097 non-null  category
 3   state      776031 non-null  category
 4   stos       767090 non-null  float32 
 5   dtos       722004 non-null  float32 
 6   tot_pkts   776097 non-null  int32   
 7   tot_bytes  776097 non-null  int32   
 8   src_bytes  776097 non-null  int32   
 9   label      776097 non-null  category
 10  Family     776097 non-null  category
dtypes: category(5), float32(3), int32(3)
memory usage: 22.2 MB


###Filling Null Values

In [ ]:
# Null Value 있는 feature -> state, stos, dtos
dataset.isnull().sum()

dur              0
proto            0
dir              0
state           66
stos          9007
dtos         54093
tot_pkts         0
tot_bytes        0
src_bytes        0
label            0
Family           0
dtype: int64

In [ ]:
dataset['state'].value_counts().to_csv('state.csv')


In [ ]:
# replace Null Value to Con
dataset['state'] = dataset.state.fillna(value='CON')

In [ ]:
# nan을 앞에 있는 것으로 채움
dataset['stos'] = dataset.stos.fillna(method='pad')
dataset['dtos'] = dataset.stos.fillna(method='pad')

### 필요없는 data 제거
1. 필요없는 열 -> 'Family'
2. 필요없는 행 -> Background

In [ ]:
dataset=dataset.drop(['Family'], axis=1) # 필요없는 정보라 삭제

In [ ]:
# label에서 background data 지우기
index=dataset[dataset['label'].str.contains('Background')].index # 'Background' 포함하는 index 찾기
dataset= dataset.drop(index)
dataset.reset_index(drop=True, inplace=True) # index 차례로 만들기

In [ ]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21412 entries, 0 to 21411
Data columns (total 10 columns):
 #   Column     Non-Null Count  Dtype   
---  ------     --------------  -----   
 0   dur        21412 non-null  float32 
 1   proto      21412 non-null  category
 2   dir        21412 non-null  category
 3   state      21412 non-null  category
 4   stos       21412 non-null  float32 
 5   dtos       21412 non-null  float32 
 6   tot_pkts   21412 non-null  int32   
 7   tot_bytes  21412 non-null  int32   
 8   src_bytes  21412 non-null  int32   
 9   label      21412 non-null  category
dtypes: category(4), float32(3), int32(3)
memory usage: 623.0 KB


In [ ]:
dataset['dtos'].value_counts()

0.0    21412
Name: dtos, dtype: int64

### One-hot-encdoer를 통해 이어붙이기
1. proto, stos, dtos 바꾸기 (논문제시)
2. dir 바꾸기
3. state ? -> 제거 (너무 많음, 331개)

In [ ]:
dataset['dir'].value_counts()

   ->    15177
  <->     6059
  <?>      152
   ?>       11
  who       11
  <-         2
Name: dir, dtype: int64

In [ ]:
# proto, stos, dtos, dir 모두 one-hot-encoder 로 변환
from sklearn.preprocessing import OneHotEncoder
# proto
ohe_proto = OneHotEncoder(sparse=False)
ohe_proto.fit(dataset[['proto']])
df_proto = pd.DataFrame(ohe_proto.transform(dataset[['proto']]), columns=['proto0', 'proto1', 'proto2','proto3'])

# stos
ohe_stos = OneHotEncoder(sparse=False)
ohe_stos.fit(dataset[['stos']])
df_stos = pd.DataFrame(ohe_stos.transform(dataset[['stos']]), columns=['stos0'])

# dtos
ohe_dtos = OneHotEncoder(sparse=False)
ohe_dtos.fit(dataset[['dtos']])
df_dtos = pd.DataFrame(ohe_dtos.transform(dataset[['dtos']]), columns=['dtos0'])

# dir
ohe_dir = OneHotEncoder(sparse=False)
ohe_dir.fit(dataset[['dir']])
df_dir = pd.DataFrame(ohe_dir.transform(dataset[['dir']]), columns=['dir0','dir1','dir2','dir3','dir4','dir5'])



In [ ]:
dataset

,dur,proto,dir,state,stos,dtos,tot_pkts,tot_bytes,src_bytes,label
0,3587.569824,tcp,<?>,PA_PA,0.0,0.0,3049,978731,245317,flow=From-Normal-V51-Grill
1,198.072739,tcp,<?>,FA_FA,0.0,0.0,14,924,462,flow=From-Normal-V51-Grill
2,197.928329,tcp,<?>,FA_FA,0.0,0.0,14,924,462,flow=From-Normal-V51-Grill
3,0.000399,udp,<->,CON,0.0,0.0,2,400,74,flow=From-Normal-V51-Stribrek
4,0.000400,udp,<->,CON,0.0,0.0,2,400,74,flow=From-Normal-V51-Stribrek
...,...,...,...,...,...,...,...,...,...,...
21407,3.955126,tcp,->,SPA_SPA,0.0,0.0,7,567,224,flow=From-Botnet-V51-1-TCP-Established
21408,0.000000,udp,->,INT,0.0,0.0,1,287,287,flow=From-Normal-V51-UDP-CVUT-DNS-Server
21409,0.000937,icmp,<->,ECO,0.0,0.0,2,196,98,flow=From-Normal-V51-Grill
21410,0.000913,icmp,<->,ECO,0.0,0.0,2,196,98,flow=From-Normal-V51-Grill


In [ ]:
# state 생략 후 one-hot-encdoer 변환된 부분 이어붙임
dataset = pd.concat([dataset[['dur']], df_proto, df_dir, df_stos, df_dtos,  dataset[['tot_pkts','tot_bytes','src_bytes','label']]], axis=1)

In [ ]:
dataset

,dur,proto0,proto1,proto2,proto3,dir0,dir1,dir2,dir3,dir4,dir5,stos0,dtos0,tot_pkts,tot_bytes,src_bytes,label
0,3587.569824,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,3049,978731,245317,flow=From-Normal-V51-Grill
1,198.072739,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,14,924,462,flow=From-Normal-V51-Grill
2,197.928329,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,14,924,462,flow=From-Normal-V51-Grill
3,0.000399,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,2,400,74,flow=From-Normal-V51-Stribrek
4,0.000400,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,2,400,74,flow=From-Normal-V51-Stribrek
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21407,3.955126,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,7,567,224,flow=From-Botnet-V51-1-TCP-Established
21408,0.000000,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1,287,287,flow=From-Normal-V51-UDP-CVUT-DNS-Server
21409,0.000937,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,2,196,98,flow=From-Normal-V51-Grill
21410,0.000913,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,2,196,98,flow=From-Normal-V51-Grill


1. hyperparameter 를 바꿔가면서 
2. 다른 dataset 활용
 -> network data에 대해서 


1. deep k-means
2. autoencdoer 먼저 학습하고 deep k-means